In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load real EEG data
real_data = pd.read_csv("C:/Users/User/Desktop/Hackathon/Impulse/output/standardized_features_noisytraindata.csv")

# Preprocess the data
X_real = real_data.drop(columns=["class"], errors="ignore").values
y_real = real_data["class"].values

# Define the GAN architecture
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(X_real.shape[1], activation='tanh'))  # Output shape matches the real data
    return model

def build_discriminator(input_shape):
    model = keras.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=input_shape))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification
    return model

# Compile the models
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator((X_real.shape[1],))

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Build the GAN
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Training the GAN
def train_gan(epochs, batch_size):
    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, X_real.shape[0], batch_size)
        real_samples = X_real[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_samples = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_samples, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        if epoch % 100 == 0:
            print(f"Epoch {epoch} | Discriminator Loss: {d_loss[0]} | Generator Loss: {g_loss}")

# Run the training
train_gan(epochs=10000, batch_size=32)
